In [1]:
!pip install folium
!pip install geopandas
!pip install shapely
!pip install geopy
!pip install geoplot
!pip install matplotlib
!pip install seaborn
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 91.6 MB/s eta 0:00:00


In [4]:
import random
import folium
from folium import PolyLine, Polygon, Marker
import zipfile
import os

In [3]:
import folium
from folium import PolyLine, Polygon, Marker
import zipfile
import os

def determinar_tipo_geometria(coordenadas):
    if isinstance(coordenadas, list):
        if len(coordenadas) == 1:
            return "Ponto"
        elif coordenadas[0] == coordenadas[-1]:
            return "Polígono"
        else:
            return "Linha"
    return "Desconhecido"

def identificar_geometria(dados):
    for item in dados:
        if 'coordenadas' in item and isinstance(item['coordenadas'], list):
            tipo_geometria = determinar_tipo_geometria(item['coordenadas'])
            item['tipo_geometria'] = tipo_geometria
        else:
            item['tipo_geometria'] = "Área"  # inferido sem coordenadas

def gerar_txt(dados, caminho_arquivo):
    with open(caminho_arquivo, 'w') as f:
        for item in dados:
            f.write(f"{item.get('id', 'sem_id')}\t{item['tipo_geometria']}\n")

def criar_mapa_pontos(dados):
    mapa = folium.Map(location=[0, 0], zoom_start=2)
    for item in dados:
        if item['tipo_geometria'] == "Ponto":
            coords = item['coordenadas'][0]
            Marker(location=coords[::-1], popup=f"ID: {item.get('id', '')}").add_to(mapa)
    return mapa

def criar_mapa_linhas(dados):
    mapa = folium.Map(location=[0, 0], zoom_start=2)
    for item in dados:
        if item['tipo_geometria'] == "Linha":
            coords = [c[::-1] for c in item['coordenadas']]  # inverte lat, lon
            PolyLine(locations=coords, color='blue').add_to(mapa)
    return mapa

def criar_mapa_poligonos(dados):
    mapa = folium.Map(location=[0, 0], zoom_start=2)
    for item in dados:
        if item['tipo_geometria'] in ["Polígono", "Área"]:
            if 'coordenadas' in item and item['tipo_geometria'] == "Polígono":
                coords = [c[::-1] for c in item['coordenadas']]
                Polygon(locations=coords, color='green', fill=True, fill_opacity=0.5).add_to(mapa)
            else:
                # Para "Área" sem coordenadas, você pode adicionar lógica específica
                pass
    return mapa

def salvar_mapas(dados, pasta):
    mapa_pontos = criar_mapa_pontos(dados)
    mapa_linhas = criar_mapa_linhas(dados)
    mapa_poligonos = criar_mapa_poligonos(dados)

    os.makedirs(pasta, exist_ok=True)
    mapa_pontos.save(os.path.join(pasta, 'mapa_pontos.html'))
    mapa_linhas.save(os.path.join(pasta, 'mapa_linhas.html'))
    mapa_poligonos.save(os.path.join(pasta, 'mapa_poligonos.html'))

def criar_zip_com_arquivos(pasta, nome_zip):
    with zipfile.ZipFile(nome_zip, 'w') as zipf:
        for root, _, files in os.walk(pasta):
            for file in files:
                caminho_completo = os.path.join(root, file)
                zipf.write(caminho_completo, arcname=file)

if __name__ == "__main__":
    dados_exemplo = [
        {"id": 1, "coordenadas": [[-23.55052, -46.633308]]},  # Ponto
        {"id": 2, "coordenadas": [[-23.55052, -46.633308], [-23.551, -46.634]]},  # Linha
        {"id": 3, "coordenadas": [[-23.55, -46.63], [-23.55, -46.62], [-23.56, -46.62], [-23.55, -46.63]]},  # Polígono
        {"id": 4},  # Área (sem coordenadas)
    ]

    identificar_geometria(dados_exemplo)

    pasta_trabalho = 'saida_geometria'
    arquivo_txt = os.path.join(pasta_trabalho, 'tipos_geometria.txt')

    os.makedirs(pasta_trabalho, exist_ok=True)
    gerar_txt(dados_exemplo, arquivo_txt)
    salvar_mapas(dados_exemplo, pasta_trabalho)

    criar_zip_com_arquivos(pasta_trabalho, 'resultado_geometria.zip')

    print("Processo finalizado. O arquivo 'resultado_geometria.zip' foi criado.")


Processo finalizado. O arquivo 'resultado_geometria.zip' foi criado.


In [5]:
# --- Geração de dados RS ---
def gerar_dados_rs(n=500):
    dados = []
    for i in range(1, n+1):
        tipo = random.choice(['ponto', 'linha', 'poligono', 'area'])
        lat_min, lat_max = -33.75, -27.5
        lon_min, lon_max = -57.5, -49.5

        if tipo == 'ponto':
            coord = [[
                round(random.uniform(lat_min, lat_max), 6),
                round(random.uniform(lon_min, lon_max), 6)
            ]]
            dados.append({'id': i, 'coordenadas': coord})

        elif tipo == 'linha':
            coord = [
                [round(random.uniform(lat_min, lat_max), 6), round(random.uniform(lon_min, lon_max), 6)],
                [round(random.uniform(lat_min, lat_max), 6), round(random.uniform(lon_min, lon_max), 6)]
            ]
            dados.append({'id': i, 'coordenadas': coord})

        elif tipo == 'poligono':
            p1 = [round(random.uniform(lat_min, lat_max), 6), round(random.uniform(lon_min, lon_max), 6)]
            p2 = [p1[0] + round(random.uniform(-0.1, 0.1), 6), p1[1] + round(random.uniform(-0.1, 0.1), 6)]
            p3 = [p2[0] + round(random.uniform(-0.1, 0.1), 6), p2[1] + round(random.uniform(-0.1, 0.1), 6)]
            poligono = [p1, p2, p3, p1]
            dados.append({'id': i, 'coordenadas': poligono})

        else:  # área sem coordenadas
            dados.append({'id': i})

    return dados

# --- Funções para identificação e processamento ---
def determinar_tipo_geometria(coordenadas):
    if isinstance(coordenadas, list):
        if len(coordenadas) == 1:
            return "Ponto"
        elif coordenadas[0] == coordenadas[-1]:
            return "Polígono"
        else:
            return "Linha"
    return "Desconhecido"

def identificar_geometria(dados):
    for item in dados:
        if 'coordenadas' in item and isinstance(item['coordenadas'], list):
            tipo_geometria = determinar_tipo_geometria(item['coordenadas'])
            item['tipo_geometria'] = tipo_geometria
        else:
            item['tipo_geometria'] = "Área"

def gerar_txt(dados, caminho_arquivo):
    with open(caminho_arquivo, 'w') as f:
        for item in dados:
            f.write(f"{item.get('id', 'sem_id')}\t{item['tipo_geometria']}\n")

def criar_mapa_pontos(dados):
    mapa = folium.Map(location=[-30.03, -51.23], zoom_start=7)
    for item in dados:
        if item['tipo_geometria'] == "Ponto":
            coords = item['coordenadas'][0]
            Marker(location=coords[::-1], popup=f"ID: {item.get('id', '')}").add_to(mapa)
    return mapa

def criar_mapa_linhas(dados):
    mapa = folium.Map(location=[-30.03, -51.23], zoom_start=7)
    for item in dados:
        if item['tipo_geometria'] == "Linha":
            coords = [c[::-1] for c in item['coordenadas']]
            PolyLine(locations=coords, color='blue').add_to(mapa)
    return mapa

def criar_mapa_poligonos(dados):
    mapa = folium.Map(location=[-30.03, -51.23], zoom_start=7)
    for item in dados:
        if item['tipo_geometria'] in ["Polígono", "Área"]:
            if 'coordenadas' in item and item['tipo_geometria'] == "Polígono":
                coords = [c[::-1] for c in item['coordenadas']]
                Polygon(locations=coords, color='green', fill=True, fill_opacity=0.5).add_to(mapa)
            else:
                # Sem coordenadas para área, pode pular ou adicionar lógica
                pass
    return mapa

def salvar_mapas(dados, pasta):
    mapa_pontos = criar_mapa_pontos(dados)
    mapa_linhas = criar_mapa_linhas(dados)
    mapa_poligonos = criar_mapa_poligonos(dados)

    os.makedirs(pasta, exist_ok=True)
    mapa_pontos.save(os.path.join(pasta, 'mapa_pontos.html'))
    mapa_linhas.save(os.path.join(pasta, 'mapa_linhas.html'))
    mapa_poligonos.save(os.path.join(pasta, 'mapa_poligonos.html'))

def criar_zip_com_arquivos(pasta, nome_zip):
    with zipfile.ZipFile(nome_zip, 'w') as zipf:
        for root, _, files in os.walk(pasta):
            for file in files:
                caminho_completo = os.path.join(root, file)
                zipf.write(caminho_completo, arcname=file)

# --- Execução principal ---
if __name__ == "__main__":
    dados_rs = gerar_dados_rs(500)
    identificar_geometria(dados_rs)

    pasta_saida = 'saida_geometria_rs'
    arquivo_txt = os.path.join(pasta_saida, 'tipos_geometria.txt')

    os.makedirs(pasta_saida, exist_ok=True)
    gerar_txt(dados_rs, arquivo_txt)
    salvar_mapas(dados_rs, pasta_saida)

    criar_zip_com_arquivos(pasta_saida, 'resultado_geometria_rs.zip')

    print("Processo concluído! O arquivo 'resultado_geometria_rs.zip' foi criado.")


Processo concluído! O arquivo 'resultado_geometria_rs.zip' foi criado.
